### build-mysql-database
CONSTROI BASE DE DADOS NO MYSQL POR MEIO DE DOCUMENTOS DA COLEÇÃO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

In [2]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re
import pymysql
#import psycopg2
import getpass

from sshtunnel import SSHTunnelForwarder
import time

import codecs
from langdetect import detect

In [3]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
define pastas de trabalho
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#path = "C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/"
#path = "C:/Users/marcelo.ribeiro/Documents/temp/" #TESTE!
#outputs = "C:/Users/marcelo.ribeiro/Documents/outputs/"
path = "/home/rsouza/Documentos/text-learning-tools/textfiles-corrected-regrouped/"

files = [f for f in os.listdir(path)]
fullpath_list = []
fullpath = ''
for file in files:
    fullpath = path+file
    fullpath_list.append(fullpath)

In [4]:
files[0:4]

['AAS_mre_rb_1974.04.17_doc_I-66.txt',
 'AAS_mre_onu_1974.03.12_doc_II-49.txt',
 'AAS_mre_onu_1974.03.12_doc_V-2.txt',
 'AAS_mre_be_1974.04.16_doc_VI-18.txt']

In [38]:
count = 0
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        #ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_private_key_password = input('Entre com a senha da chave privada'),
                        #ssh_username="marcelobribeiro",
                        ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           #user='marcelobribeiro', 
                           user='rsouza',
                           passwd=pass_mysql,
                           #passwd=input('Entre com a senha do usuário no banco de dados'),
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    #sql = "select * from marcelo_draft"
    #cur.execute(sql)
    #cur.execute("USE CPDOC_AS")
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere id de documentos e corpo de texto na base mysql
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    cur.execute('''CREATE TABLE IF NOT EXISTS marcelo_draft 
               (id VARCHAR(50), body LONGTEXT, date VARCHAR(20), readability TINYINT(2), main_language VARCHAR(10)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')
    #cur.execute('INSERT INTO marcelo_draft VALUES ("1", "2", "3", "4", "5")')
    cur.execute("SELECT * FROM marcelo_draft")
    for txt in fullpath_list[0:1]:
        count += 1
        txt_date = re.sub('.*(19\d\d.*)_.*_.*', r'\1', txt)
        txt_id = re.sub('.*AAS_mre_(.*).txt', r'\1', txt)
        #obs: deveria usar encoding utf-8. Ignorando erros
        with open(txt, 'r', encoding='ISO-8859-1') as f:
            txt_body = f.read()
            #txt_body = txt_body.decode('latin-1')#.encode('utf-8')
            #txt_body = txt_body.encode('latin-1')
            #txt_body = txt_body.decode('utf-8')
            print(txt_body)
            #cur.execute('''INSERT INTO marcelo_draft VALUES (txt_id, txt_body, txt_date)''')
            cur.execute("INSERT INTO marcelo_draft VALUES (%s), (%s), (%s), (%s), (%s))" % (txt_id, txt_body, txt_date, 0, 0))
            #cur.execute('''INSERT INTO marcelo_draft VALUES {} {} {} {} {}'''.format(txt_id, txt_body, txt_date, 0, 0))
            #cur.execute("INSERT INTO marcelo_draft VALUES (?,?,?,?,?)",(txt_id, txt_body, txt_date, "0", "0"))
    #cur.execute("SELECT * FROM marcelo_draft");
    #rows = cur.fetchall()
    #print(rows)

········
········
 

aas, vãqú oh 4?
/y, ', - -- (, ( rio

, brasília, em 29 de dezembro de 1976

sua excelência o senhor

embaixador enrique pérez-hernández
embaixada da espanha

buenos aires - argentina

prezado embaixador e caro amigo,

foi com grande alegria que recebi sua?
carta de 7 de dezembro corrente com a notícia de que havia assumido a embaixada da espanha em buenos aires.

como bem sabe, por cinco anos tive a
grande honra de representar meu país na argentina e o
prazer de gozar da convivência dos amigos argentinos. _

ainda tenho vivas as lembranças dos :

 

tempos de genebra e espero um dia voltar a vê-lo, seja

vaqui em brasília seja alhures.

 

 

cordial abraça, "

tam"

xv w

(an nio f. a edo da silveira)

(a.

lpt

 

mé mu . ou. i:!-
wl "

buenos aires, 7 de diciembre de 1976.

(o 4 . (o .
klmdnqaôox nx" &). "ismm

excmo. sr. don antonio francisco azeredo da silveira,
ministro de relaciones exteriores,

b r a s i l i a.

mi querido ministro y amigo:

ayer presenté 

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'oh 4?\n/y, ', - -- (, ( rio\n\n, brasília, em 29 de dezembro de 1976\n\nsua excelên' at line 3")

In [ ]:
for txt in fullpath_list:
    txt_date = re.sub('.*(19\d\d.*)_.*_.*', r'\1', txt)
    txt_id = re.sub('.*AAS_mre_(.*).txt', r'\1', txt)
    with open(os.path.join(path,txt), 'r', encoding='utf-8') as f:
        txt_body = f.read()
        #curs.execute('''INSERT INTO marcelo_draft VALUES (txt_id, txt_body, txt_date, txt_date)''')

In [58]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Classifica o corpus de acordo com: tipo textual (para separar textos sujos, como letra cursiva, foto, fonte alternativa) e idioma
pt = 1
de = 2
en = 3
es = 4
fr = 5
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
doc_class = []
lang_class = []
count_doc = 0
#percentil = int(len(onlyfiles)/100)
for txt in fullpath_list[0:]:
    #print(txt)
    with open(txt, 'r', encoding='utf-8') as f:
        #if count_doc % percentil == 0: print(int(count_doc/percentil),'% done')
        #if count_doc % (percentil-1) == 0: clear_output()
        texto = f.read()
        texto_pt = texto_en = texto_es = texto_fr = texto
        #texto_split = re.split('\.|\[|\]|\(|\)|\?|\/|\n|\*|\:|\%|\d+|\ª|\º', texto)
        texto_split = re.split('\.|\?|\:', texto)
        #print(texto_split)
        count = pt_count = en_count = es_count = fr_count = de_count = 0
        total_count = 0
        for phrase in texto_split:
            total_count += 1
            if not re.search('[^\W\d_]', phrase): continue
            language = detect(phrase)
            #print('PHRASE', phrase, 'LANGUAGE', language)
            total_count += 1
            if language == 'pt':
                pt_count += 1
            if language == 'en':
                en_count += 1
            if language == 'es':
                es_count += 1
            if language == 'fr':
                fr_count += 1
            if language == 'de':
                de_count += 1
        lang_count = pt_count + en_count + es_count + fr_count + de_count        
        #print(total_count, lang_count, 'proportion', lang_count/total_count)
        if lang_count/total_count < 0.03: 
            doc_class.append(0)
        else: doc_class.append(1)
        lang_class.append(0)
        if de_count/total_count > 0.1: 
            lang_class[-1] = 2
        if fr_count/total_count > 0.1: 
            lang_class[-1] = 5
        if es_count/total_count > 0.1: 
            lang_class[-1] = 4
        if en_count/total_count > 0.1: 
            lang_class[-1] = 3
        if pt_count/total_count > 0.1: 
            lang_class[-1] = 1
        #else: 

        #print(pt_count/total_count)
        #print(en_count/total_count)
        #print(es_count/total_count)
        #print(fr_count/total_count)
        #print(de_count/total_count)
        
        #print('len',len(texto_split))
        #print(doc_class)
        

### RASCUNHOS

In [28]:
count = 0
for txt in fullpath_list[0:1]:
        count += 1
        txt_date = re.sub('.*(19\d\d.*)_.*_.*', r'\1', txt)
        txt_id = re.sub('.*AAS_mre_(.*).txt', r'\1', txt)
        #obs: deveria usar encoding utf-8
        with open(txt, 'rb') as f:
            print(f)
            txt_body = f.read()
            
            txt_body = txt_body.decode('latin-1').encode('utf8')
            print(txt_body)

<_io.BufferedReader name='C:/Users/marcelo.ribeiro/Documents/temp/AAS_mre_ag_1973.11.20_doc_I-1.txt'>
b' \r\n\r\njr, /h \' "& m$ hb\'h\'qo\r\n& la\'?\r\n\r\n, \xc3\xa9 %%3 - _\r\n(aonsulado do brasil _/ , ova/caio\r\n4 & ,\r\n* brazilian consulate wo \'\r\naas hawthorn st. 16 % wja mm\r\nmu. .. " "\r\nmm .\r\n\r\nnew bedford, massachuseti\'s. u. s, a, 02740\r\n\r\n \r\n\r\nmarcio muller bueno\r\n" , /\r\n11:11:3sz mao jz". www n\xc3\xa3o"3337552 %%, " aca-m\r\n)\r\nwha %" wom- aba/v\xc3\xa0 %%\r\n\r\nxb" 7/35 andado.\r\n\r\n_, \xc3\xad\xc3\xa1aw m\xc3\xad. m luz/\xc3\xa1 && %% %%, aj alma, ;\r\n(\r\n\r\n"me/&& % \'wl-(aajj-mi- # "a" aaa/fo &\r\n"n\xc3\xa3o. aiwa %%%&/a %%%&/www a* m" \xc3\xa1?!" \xc3\xa1\r\n1/1"me %"a\xc3\xad\'ej\'wfo m/k%/\xc3\xa1m%\xc3\xa2 tai/\xc3\xa1\r\nw. : \xc3\xbaw\xc3\xa1 2-- /i < %%\r\n"melwa/ vol\xc3\xadco ay? mul/1 "( o , (-\r\n\r\n\' % \xc3\xb3<\r\n/l ano)/4% wklt-w wa: (lw/\xc3\xa1m/[a w\r\n&& mwr\xc3\xa1ww/l\xc3\xa1w wa",\r\n\r\n/-\r\n\r\n\r\n"ac-"44. \' 

In [14]:

codecs.decode('''jr, /h ' "& m$ hb'h'qo & la'?  , é %%3 - _ (aonsulado do brasil _/ , ova/caio 4 & , * brazilian consulate wo ' aas hawthorn st. 16 % wja mm mu. .. " " mm .  new bedford, massachuseti's. u. s, a, 02740     marcio muller bueno " , / 11:11:3sz mao jz". www não"3337552 %%, " aca-m ) wha %" wom- aba/và %%  xb" 7/35 andado.  _, íáaw mí. m luz/á && %% %%, aj alma, ; (  "me/&& % 'wl-(aajj-mi- # "a" aaa/fo & "não. aiwa %%%&/a %%%&/www a* m" á?!" á 1/1"me %"aí'ej'wfo m/k%/ám%â tai/á w. : úwá 2-- /i < %% "melwa/ volíco ay? mul/1 "( o , (-  ' % ó< /l ano)/4% wklt-w wa: (lw/ám/[a w && mwráww/láw wa",  /-   "ac-"44. ' pá , (". v £ f (1. é : "l 01; mw w wm %- & /  wl] [iv-156%; , (, ' "ab/q "já-m m€) kífa-nmc aaa-x mw, ala www, %"wáwí, 9 m: ( c/mn  %sjjéãm  "ém-k w [', /á" "**-"í; pé ifá?!" w %'0 m amo. .. /lilia"? jmá%. %i/w % && po "vpmtãvak_ jun moo-"n & %% "4/4, "  p (aõ piatã h , azo"wat ao.     the secretary of state washington  november 20, 1973  dear mr. bueno:  thank you for your letter of congratulations. i remember our work with the brazilian student-leaders, and i deeply appreciate hearing from you.  best regards,  por"?  henry a. Kissinger  mr. marcio muller bueno, consul of brazil, brazilian consulate, 229 hawthorn street, new bedford, massachusetts.  ;" m ''')
#'\xc4pple'
#apple.decode('iso-8859-1').encode('utf8')

TypeError: decoding with 'utf-8' codec failed (TypeError: a bytes-like object is required, not 'str')